In [1]:
!pip install langchain langgraph

  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.99
    Uninstalling langsmith-0.1.99:
      Successfully uninstalled langsmith-0.1.99
  Attempting uninstall: langchain-core━━━━━━━━━━━━━━━━━━━━━━━━ 3/8 [langgraph-sdk]
    Found existing installation: langchain-core 0.2.33━━━━━━━━ 3/8 [langgraph-sdk]
    Uninstalling langchain-core-0.2.33:━━━━━━━━━━━━━━━━━━━━━━━ 3/8 [langgraph-sdk]
      Successfully uninstalled langchain-core-0.2.33━━━━━━━━━━ 3/8 [langgraph-sdk]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [langgraph]/8 [langgraph]checkpoint]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-aws 0.1.16 requires boto3<1.35.0,>=1.34.131, but you have boto3 1.38.5 which is incompatible.
langchain-community 0.2.12 requires langchain<0.3.0,>=0.2.13, but you have langchain 0.2.5 which is incompatible.


In [9]:
import os
from dotenv import load_dotenv
from langchain_aws import ChatBedrock, ChatBedrockConverse
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage
from botocore.config import Config
import boto3
import ipywidgets as widgets

In [10]:
# Load environment variables
load_dotenv()
aws_region = os.getenv("AWS_REGION")

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client in your aws region.
# If you do not have the AWS CLI profile setup, you can authenticate with aws access key, secret and session token.
# For more details check https://docs.aws.amazon.com/cli/v1/userguide/cli-authentication-short-term.html
bedrock_rt = boto3.client(
    "bedrock-runtime",
    region_name=aws_region,
    config=bedrock_config,
)


In [11]:
model_selection = widgets.Dropdown(
    options=[
        'us.meta.llama4-maverick-17b-instruct-v1:0',
        'us.meta.llama4-scout-17b-instruct-v1:0'
    ],
    value='us.meta.llama4-maverick-17b-instruct-v1:0',
    description='Select model:',
    disabled=False,
)
model_selection

Dropdown(description='Select model:', options=('us.meta.llama4-maverick-17b-instruct-v1:0', 'us.meta.llama4-sc…

In [12]:
model_id = model_selection.value
model_kwargs={
        "temperature": 0,
        "top_p": 0.9,
        "max_tokens": 1024
    }
llm = ChatBedrockConverse(
    model=model_id,
    **model_kwargs
)

In [13]:
# Setup memory
memory = MemorySaver()

# Define a simple tool for testing
@tool
def calculator(expression: str) -> str:
    """Calculate the result of a mathematical expression.
    
    Args:
        expression: The mathematical expression to calculate
    """
    print(f"CALCULATOR TOOL CALLED with: {expression}")
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

In [14]:
system_message = SystemMessage(content="""You are a helpful assistant that always uses tools when they are available.
When asked a question that requires calculation, ALWAYS use the calculator tool instead of doing the calculation yourself.
NEVER try to solve math problems directly - ALWAYS use the calculator tool.""")

In [15]:
tools = [calculator]
graph = create_react_agent(
    llm, 
    tools=tools,
    checkpointer=memory,
    prompt=system_message
)

In [16]:

# Test the agent
inputs = {"messages": [HumanMessage(content="What is 25 * 32?")]}
config = {"configurable": {"thread_id": "test_thread"}}

# Run the agent and print the response
print("Starting agent execution with Llama + system message...")
for s in graph.stream(inputs, config, stream_mode="values"):
    message = s["messages"][-1]
    if hasattr(message, "pretty_print"):
        message.pretty_print()
    else:
        print(message)

Starting agent execution with Llama + system message...
================================ Human Message =================================

What is 25 * 32?
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'calculator', 'input': {'expression': '25 * 32'}, 'id': 'tooluse_FhizmiwqRtiKjgNKCN6rPA'}]
Tool Calls:
  calculator (tooluse_FhizmiwqRtiKjgNKCN6rPA)
 Call ID: tooluse_FhizmiwqRtiKjgNKCN6rPA
  Args:
    expression: 25 * 32
CALCULATOR TOOL CALLED with: 25 * 32
================================= Tool Message =================================
Name: calculator

The result of 25 * 32 is 800
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'calculator', 'input': {'expression': '25 * 32'}, 'id': 'tooluse_wNPfM4W-TJuI8tlISDtEOQ'}]
Tool Calls:
  calculator (tooluse_wNPfM4W-TJuI8tlISDtEOQ)
 Call ID: tooluse_wNPfM4W-TJuI8tlISDtEOQ
  Args:
    expression: 25 * 32
CALCULATOR T